# 09: GNSS Scenario Generation

This workshop covers multi-satellite GNSS IQ signal generation with realistic
propagation effects including orbital Doppler, atmospheric delays, and multipath.

## Learning Objectives
- Understand GNSS signal structure (GPS L1 C/A, Galileo E1)
- Generate composite multi-satellite IQ signals
- Analyze orbital geometry effects (Doppler, path loss)
- Apply ionospheric and tropospheric delay models
- Run PCPS acquisition on generated scenarios

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import struct
import os

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

# Helper: run r4w CLI and capture output
def r4w(*args):
    result = subprocess.run(['cargo', 'run', '--bin', 'r4w', '--'] + list(args),
                          capture_output=True, text=True, cwd=os.path.join(os.getcwd(), '..'))
    print(result.stdout)
    if result.returncode != 0:
        print('STDERR:', result.stderr)
    return result

# Helper: read raw IQ file (interleaved f64)
def read_iq(path):
    with open(path, 'rb') as f:
        data = f.read()
    samples = struct.unpack(f'<{len(data)//8}d', data)
    return np.array(samples[0::2]) + 1j * np.array(samples[1::2])

## Part 1: GNSS Signal Basics

GNSS satellites transmit CDMA signals at known frequencies. Each satellite has a
unique PRN (Pseudo-Random Noise) code that enables code-division multiple access.

| Signal | Frequency | Chipping Rate | Code Length | Modulation |
|--------|-----------|--------------|-------------|------------|
| GPS L1 C/A | 1575.42 MHz | 1.023 Mchip/s | 1023 chips | BPSK(1) |
| GPS L5 | 1176.45 MHz | 10.23 Mchip/s | 10230 chips | QPSK(10) |
| GLONASS L1OF | ~1602 MHz | 0.511 Mchip/s | 511 chips | BPSK(0.5) |
| Galileo E1 | 1575.42 MHz | 1.023 Mchip/s | 4092 chips | CBOC |

Let's explore these with the CLI:

In [ ]:
# Show all GNSS signal parameters
r4w('gnss', 'compare')

## Part 2: Generating a GNSS Scenario

A GNSS scenario models multiple satellites simultaneously transmitting to a
ground receiver. The composite signal includes:

1. **Per-satellite PRN code** at the correct code phase (from geometric range)
2. **Doppler shift** from satellite-receiver relative motion (~+/-5 kHz)
3. **Path loss** from ~20,200 km distance (~182 dB)
4. **Ionospheric delay** (1/f^2 group delay, ~5-15m at L1)
5. **Tropospheric delay** (elevation-dependent, ~2.3m zenith)
6. **Thermal noise** (kTB at receiver input)

In [ ]:
# List available scenario presets
r4w('gnss', 'scenario', '--list-presets')

In [ ]:
# Generate an open-sky scenario (4 GPS SVs, 1ms, 2.046 MHz)
r4w('gnss', 'scenario', '--preset', 'open-sky',
    '--duration', '0.002',
    '--output', '/tmp/gnss_opensky.iq')

In [ ]:
# Load and visualize the generated IQ
iq = read_iq('/tmp/gnss_opensky.iq')
sample_rate = 2.046e6
t_ms = np.arange(len(iq)) / sample_rate * 1000

fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Time domain (first 0.5 ms)
mask = t_ms < 0.5
axes[0].plot(t_ms[mask], iq[mask].real, alpha=0.7, label='I')
axes[0].plot(t_ms[mask], iq[mask].imag, alpha=0.7, label='Q')
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Composite GNSS IQ (4 GPS L1 C/A satellites + noise)')
axes[0].legend()

# Power spectral density
fft_size = 4096
spectrum = np.fft.fftshift(np.fft.fft(iq[:fft_size], fft_size))
freqs = np.linspace(-sample_rate/2, sample_rate/2, fft_size) / 1e3
power_db = 10 * np.log10(np.abs(spectrum)**2 / fft_size + 1e-30)
axes[1].plot(freqs, power_db)
axes[1].set_xlabel('Frequency (kHz)')
axes[1].set_ylabel('PSD (dB)')
axes[1].set_title('Power Spectral Density')

# IQ scatter (constellation)
axes[2].scatter(iq[:2000].real, iq[:2000].imag, s=1, alpha=0.3)
axes[2].set_xlabel('I')
axes[2].set_ylabel('Q')
axes[2].set_title('IQ Scatter (noise-like for composite CDMA)')
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()

print(f'Signal statistics:')
print(f'  Samples: {len(iq)}')
print(f'  Mean power: {10*np.log10(np.mean(np.abs(iq)**2)):.1f} dB')
print(f'  Peak/avg: {10*np.log10(np.max(np.abs(iq)**2)/np.mean(np.abs(iq)**2)):.1f} dB')

## Part 3: Multi-Constellation Scenarios

Modern receivers process signals from multiple constellations simultaneously.
Let's generate a GPS + Galileo scenario and compare:

In [ ]:
# Generate multi-constellation scenario (GPS + Galileo)
r4w('gnss', 'scenario', '--preset', 'multi-constellation',
    '--duration', '0.002',
    '--output', '/tmp/gnss_multi.iq')

In [ ]:
# Compare open sky vs multi-constellation
iq_opensky = read_iq('/tmp/gnss_opensky.iq')
iq_multi = read_iq('/tmp/gnss_multi.iq')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, data, title in [
    (axes[0], iq_opensky, 'Open Sky (4 GPS)'),
    (axes[1], iq_multi, 'Multi-Constellation (3 GPS + 3 Galileo)'),
]:
    spectrum = np.fft.fftshift(np.fft.fft(data[:4096], 4096))
    freqs = np.linspace(-sample_rate/2, sample_rate/2, 4096) / 1e3
    ax.plot(freqs, 10*np.log10(np.abs(spectrum)**2/4096 + 1e-30))
    ax.set_xlabel('Frequency (kHz)')
    ax.set_ylabel('PSD (dB)')
    ax.set_title(title)

plt.tight_layout()
plt.show()

## Part 4: Urban Canyon Scenario

In urban environments, buildings cause:
- **Multipath**: Reflected signals arrive with delay and phase offset
- **Signal blockage**: Satellites below elevation mask are blocked
- **Reduced C/N0**: Additional path loss through/around buildings

Let's compare open sky vs urban canyon:

In [ ]:
# Generate urban canyon scenario
r4w('gnss', 'scenario', '--preset', 'urban-canyon',
    '--duration', '0.002',
    '--output', '/tmp/gnss_urban.iq')

In [ ]:
# Compare environments: time domain
iq_urban = read_iq('/tmp/gnss_urban.iq')

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

n_show = 500
t_us = np.arange(n_show) / sample_rate * 1e6  # microseconds

axes[0].plot(t_us, iq_opensky[:n_show].real, alpha=0.8)
axes[0].set_title('Open Sky - Clean composite signal')
axes[0].set_ylabel('I amplitude')

axes[1].plot(t_us, iq_urban[:n_show].real, alpha=0.8, color='orange')
axes[1].set_title('Urban Canyon - Multipath distortion')
axes[1].set_xlabel('Time (us)')
axes[1].set_ylabel('I amplitude')

plt.tight_layout()
plt.show()

# Power comparison
print(f'Open Sky power:    {10*np.log10(np.mean(np.abs(iq_opensky)**2)):.1f} dB')
print(f'Urban Canyon power: {10*np.log10(np.mean(np.abs(iq_urban)**2)):.1f} dB')

## Part 5: Output Formats & Precise Ephemeris

R4W supports multiple IQ output formats for compatibility with different SDR tools:

```bash
# Default: complex float64 (16 bytes/sample)
r4w gnss scenario --preset open-sky --output scenario.iq

# USRP/Ettus compatible: interleaved float32 (8 bytes/sample)
r4w gnss scenario --preset open-sky --format ettus --output usrp.iq

# Compact: signed int16 (4 bytes/sample)
r4w gnss scenario --preset open-sky --format sc16 --output compact.iq
```

For cm-level accuracy, use SP3 precise ephemeris and IONEX TEC maps (requires `ephemeris` feature):

```bash
r4w gnss scenario --preset open-sky \
    --sp3 /path/to/COD0OPSFIN_20260050000_01D_05M_ORB.SP3 \
    --ionex /path/to/COD0OPSFIN_20260050000_01D_01H_GIM.INX \
    --output precise.iq
```

See **Workshop 11: Precise Ephemeris** for full SP3/IONEX coverage.

## Exercises

1. **Acquisition challenge**: Load a generated scenario and implement PCPS acquisition
   to detect which PRNs are present and estimate their Doppler.

2. **SNR estimation**: Calculate the per-satellite C/N0 from the composite signal.
   Hint: Cross-correlate with the known PRN code to isolate each satellite.

3. **Doppler analysis**: Generate a high-dynamics scenario and plot how the
   Doppler shift varies across satellites.

4. **Duration sweep**: Generate scenarios of 1ms, 10ms, 100ms and observe how
   file size and generation time scale.

5. **Precise ephemeris**: If you have SP3/IONEX files, regenerate the open-sky
   scenario with `--sp3` and `--ionex` options. Compare the satellite positions
   and ionospheric delays.

In [ ]:
# Your code here
